In [3]:
# Image processing and analysis
import cv2
import numpy as np
from skimage import io, filters, segmentation, measure, morphology
from skimage.transform import resize

# Deep learning and neural networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# Data manipulation and analysis
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Other
import os
import random

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

In [1]:
def load_wound_image_and_contour(image_path, contour_data, assessment_id, patient_id):
    image = cv2.imread(image_path)
    contour_coords = np.array(contour_data)
    return image, contour_coords, assessment_id, patient_id

In [ ]:
def denormalize_contour_coordinates(contour_coords, image_width, image_height):
    denormalized_contour_coords = []
    for coord in contour_coords:
        x = int(coord[0] * image_width)
        y = int(coord[1] * image_height)
        denormalized_contour_coords.append((x, y))
    return denormalized_contour_coords

In [ ]:
# Crop Image & Translate wound contour to center
def crop_wound_image_and_transform_contour(image, contour_coords):
    # Calculate the bounding rectangle of the contour
    x, y, w, h = cv2.boundingRect(contour_coords)
    # Crop the image
    cropped_image = image[y:y+h, x:x+w]
    # Transform the contour coordinates
    transformed_contour_coords = contour_coords - [x, y]
    return cropped_image, transformed_contour_coords

In [ ]:
def calculate_contour_area(contour_coords):
    n = len(contour_coords)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += contour_coords[i][0] * contour_coords[j][1]
        area -= contour_coords[j][0] * contour_coords[i][1]
    area = abs(area) / 2.0
    return area

In [ ]:
def detect_aruco_marker(image):
    """
    Detect ArUco markers in the image and return the marker corners and IDs.
    """
    # Load the ArUco dictionary and parameters
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_250)
    aruco_params = cv2.aruco.DetectorParameters_create()
    
    # Detect markers
    corners, ids, _ = cv2.aruco.detectMarkers(image, aruco_dict, parameters=aruco_params)
    
    if ids is not None:
        # Draw markers on the image for visualization
        detected_markers_image = cv2.aruco.drawDetectedMarkers(image.copy(), corners, ids)
        return corners, ids, detected_markers_image

In [ ]:


def create_image_mask_with_contour(image, contour_coords):
    # Create a mask image with the same shape as the original image
    mask_image = np.zeros_like(image)
    # Draw the contour on the mask image
    cv2.drawContours(mask_image, [contour_coords], -1, (255, 255, 255), -1)
    return mask_image

In [ ]:
def process_wound_image(image_path, contour_data, assessment_id, patient_id, image_width, image_height):
    # Load the wound image and contour
    image, contour_coords, assessment_id, patient_id = load_wound_image_and_contour(
        image_path, contour_data, assessment_id, patient_id
    )
    
    # Denormalize the contour coordinates
    contour_coords = denormalize_contour_coordinates(contour_coords, image_width, image_height)
    
    # Crop the image and transform the contour
    cropped_image, transformed_contour_coords = crop_wound_image_and_transform_contour(
        image, contour_coords
    )
    
    return cropped_image, transformed_contour_coords, assessment_id, patient_id

### Wound Image Preprocessing - Contrast Ehhancement & Denoising 


In [ ]:
def resize_image(image):
    resized_image = cv2.resize(image, (224, 224))
    return resized_image

def convert_to_grayscale(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray_image

def convert_to_hsv(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    return hsv_image

In [ ]:
# Contrast Enhancements

def apply_logarithmic_transformation(image):
    # Logarithmic Transformation for contrast enhancement
    c = 255 / np.log(1 + np.max(image))
    log_image = c * (np.log(1 + image.astype(np.float64)))
    log_image = np.array(log_image, dtype=np.uint8)
    return log_image

def apply_contrast_stretching(image):
    # Contrast Stretching
    min_val, max_val = np.min(image), np.max(image)
    stretched_image = 255 * (image - min_val) / (max_val - min_val)
    stretched_image = np.array(stretched_image, dtype=np.uint8)
    return stretched_image

def apply_histogram_equalization_clahe(image):
    # Histogram Equalization using CLAHE
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    image_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)
    return image_clahe



In [ ]:
# Denoising

def apply_median_filter(image, kernel_size=5):
    # Median Filter for noise reduction
    return cv2.medianBlur(image, kernel_size)

def apply_gaussian_filter(image, kernel_size=5):
    # Gaussian Filter for noise reduction
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

# Convolution and Morphological Transformations

def apply_custom_convolution(image, kernel):
    # Custom Convolution operation using a kernel
    return cv2.filter2D(image, -1, kernel)

def apply_morphological_operations(image, operation='erode', kernel_size=5):
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    if operation == 'erode':
        return cv2.erode(image, kernel, iterations=1)
    elif operation == 'dilate':
        return cv2.dilate(image, kernel, iterations=1)
    return image


In [ ]:
# Image Normalization

def normalize_image(image):
    # Normalize image to [0, 0, 0] - [255, 255, 255] range
    norm_image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    return np.array(norm_image, dtype=np.uint8)



In [ ]:
# imtegrating image processing steps
def process_image(image):
    # Contrast Enhancements
    log_image = apply_logarithmic_transformation(image)
    stretched_image = apply_contrast_stretching(log_image)
    clahe_image = apply_histogram_equalization_clahe(stretched_image)

    # Denoising
    median_filtered = apply_median_filter(clahe_image)
    gaussian_filtered = apply_gaussian_filter(median_filtered)

    # Convolution
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32)  # Sharpening Kernel
    convolved_image = apply_custom_convolution(gaussian_filtered, kernel)

    # Morphological Transformation (e.g., Erosion followed by Dilation)
    eroded_image = apply_morphological_operations(convolved_image, operation='erode')
    dilated_image = apply_morphological_operations(eroded_image, operation='dilate')

    # Normalize the image to the [0, 0, 0] - [255, 255, 255] range
    final_image = normalize_image(dilated_image)

    return processed_image

In [ ]:
def active_contour_segmentation(image,contour):
    # Initialize the active contour
    init_contour = np.array(contour)
    # Apply active contour segmentation
    contour = active_contours(image, init_contour, alpha=0.1, beta=1, gamma=0.01)
    # Create a mask from the contour
    mask = find_boundaries(contour, mode='thick')
    return mask.astype('uint8') * 255

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

def segment_wound(image):
    # Apply thresholding to segment the wound
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return thresh

def classify_tissue(image):
    # Load the pre-trained model
    model = load_model('tissue_classification_model.h5')
    # Preprocess the image
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    # Make predictions
    predictions = model.predict(image)
    return predictions

def classify_wound_type(image):
    # Load the pre-trained model
    model = load_model('wound_type_classification_model.h5')
    # Preprocess the image
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    # Make predictions
    predictions = model.predict(image)
    return predictions

### WOund Image Segmentation using Transfer 

In [ ]:
# Define the Transfer Learning model using EfficientNet
def efficientnet_model(input_shape):
    base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[base_model.input], outputs=[outputs])
    return model




In [ ]:
# Define the Transfer Learning model using MobileNet
def mobilenet_model(input_shape):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[base_model.input], outputs=[outputs])
    return model


In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.applications import ResNet50, EfficientNetB7, MobileNetV2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D

# Define the U-Net model
def unet_model(input_shape):
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    drop4 = tf.keras.layers.Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    drop5 = tf.keras.layers.Dropout(0.5)(conv5)

    up6 = Conv2D(256, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = Concatenate()([drop4, up6])
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(merge6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = Conv2D(128, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = Concatenate()([conv3, up7])
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = Conv2D(64, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = Concatenate()([conv2, up8])
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = Conv2D(32, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = Concatenate()([conv1, up9])
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(merge9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = Conv2D(2, (3, 3), activation='relu', padding='same')(conv9)
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    return model



In [ ]:
# Define the R-CNN model
def rcnn_model(input_shape):
    inputs = Input(shape=input_shape)
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[base_model.input], outputs=[outputs])
    return model

In [ ]:
# Define the ensemble model
def ensemble_model(input_shape):
    unet = unet_model(input_shape)
    rcnn = rcnn_model(input_shape)
    efficientnet = efficientnet_model(input_shape)
    mobilenet = mobilenet_model(input_shape)
    
    inputs = Input(shape=input_shape)
    unet_output = unet(inputs)
    rcnn_output = rcnn(inputs)
    efficientnet_output = efficientnet(inputs)
    mobilenet_output = mobilenet(inputs)
    
    outputs = tf.keras.layers.Average()([unet_output, rcnn_output, efficientnet_output, mobilenet_output])
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [ ]:
def process_wound_image(
    image_path, 
    contour_data, 
    assessment_id, 
    patient_id, 
    image_width, 
    image_height
):
    # Load the wound image and contour
    image, contour_coords, assessment_id, patient_id = load_wound_image_and_contour(
        image_path, contour_data, assessment_id, patient_id
    )
    
    # Denormalize the contour coordinates
    contour_coords = denormalize_contour_coordinates(contour_coords, image_width, image_height)
    
    # Crop the image and transform the contour
    cropped_image, transformed_contour_coords = crop_wound_image_and_transform_contour(
        image, contour_coords
    )
    
    # Apply image processing steps
    processed_image = process_image(cropped_image)
    
    # Apply active contour segmentation
    active_contour = active_contours(processed_image, transformed_contour_coords)
    
    # Detect ArUco marker
    corners, ids, detected_markers_image = detect_aruco_marker(processed_image)
    
    # Calculate ArUco marker area (1 square cm)
    aruco_marker_area = calculate_contour_area(corners[0])
    
    # Create image mask with active contour
    mask_image = create_image_mask_with_contour(processed_image, active_contour)
    
    # Reload trained U-Net and R-CNN models
    unet_model = tf.keras.models.load_model('unet_model.h5')
    rcnn_model = tf.keras.models.load_model('rcnn_model.h5')
    
    # Extract features using U-Net and R-CNN transfer learning
    unet_features = extract_features_unet(processed_image, unet_model)
    rcnn_features = extract_features_rcnn(processed_image, rcnn_model)
    
    # Combine features
    combined_features = np.concatenate([unet_features, rcnn_features], axis=1)
    
    # Reload trained tissue and color classification model
    classification_model = tf.keras.models.load_model('classification_model.h5')
    
    # Perform tissue and color classification
    tissue_color_classification = classify_tissue_color(combined_features, classification_model)
    
    # Reload trained wound classification model
    wound_classification_model = tf.keras.models.load_model('wound_classification_model.h5')
    
    # Perform wound classification
    wound_classification = classify_wound(combined_features, wound_classification_model)
    
    # Calculate wound surface area (in square cm)
    wound_surface_area = calculate_contour_area(active_contour) / aruco_marker_area
    
    return (
        wound_surface_area, 
        detected_markers_image, 
        mask_image, 
        processed_image, 
        tissue_color_classification, 
        wound_classification
    )

def extract_features_unet(image, model):
    # Preprocess image for U-Net input
    image = preprocess_image(image, model.input_shape[1:])
    
    # Extract features using U-Net
    features = model.predict(image)
    
    return features

def extract_features_rcnn(image, model):
    # Preprocess image for R-CNN input
    image = preprocess_image(image, model.input_shape[1:])
    
    # Extract features using R-CNN
    features = model.predict(image)
    
    return features

def classify_tissue_color(features, model):
    # Classify tissue and color
    classification = model.predict(features)
    
    return classification

def classify_wound(features, model):
    # Classify wound
    classification = model.predict(features)
    
    return classification

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

def segment_wound(image):
    # Apply thresholding to segment the wound
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return thresh

def classify_tissue(image):
    # Load the pre-trained model
    model = load_model('tissue_classification_model.h5')
    # Preprocess the image
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    # Make predictions
    predictions = model.predict(image)
    return predictions

def classify_wound_type(image):
    # Load the pre-trained model
    model = load_model('wound_type_classification_model.h5')
    # Preprocess the image
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    # Make predictions
    predictions = model.predict(image)
    return predictions

In [ ]:
def process_wound_image(
    image_path, 
    contour_data, 
    assessment_id, 
    patient_id, 
    image_width, 
    image_height
):
    # Load the wound image and contour
    image, contour_coords, assessment_id, patient_id = load_wound_image_and_contour(
        image_path, contour_data, assessment_id, patient_id
    )
    
    # Denormalize the contour coordinates
    contour_coords = denormalize_contour_coordinates(contour_coords, image_width, image_height)
    
    # Crop the image and transform the contour
    cropped_image, transformed_contour_coords = crop_wound_image_and_transform_contour(
        image, contour_coords
    )
    
    # Apply image processing steps
    processed_image = process_image(cropped_image)
    
    # Apply active contour segmentation
    active_contour = active_contours(processed_image, transformed_contour_coords)
    
    # Detect ArUco marker
    corners, ids, detected_markers_image = detect_aruco_marker(processed_image)
    
    # Calculate ArUco marker area (1 square cm)
    aruco_marker_area = calculate_contour_area(corners[0])
    
    # Create image mask with active contour
    mask_image = create_image_mask_with_contour(processed_image, active_contour)
    
    # Extract features using U-Net and R-CNN transfer learning
    unet_features = extract_features_unet(processed_image)
    rcnn_features = extract_features_rcnn(processed_image)
    
    # Combine features
    combined_features = np.concatenate([unet_features, rcnn_features], axis=1)
    
    # Perform tissue and color classification
    tissue_color_classification = classify_tissue_color(combined_features)
    
    # Perform wound classification
    wound_classification = classify_wound(combined_features)
    
    # Calculate wound surface area (in square cm)
    wound_surface_area = calculate_contour_area(active_contour) / aruco_marker_area
    
    return (
        wound_surface_area, 
        detected_markers_image, 
        mask_image, 
        processed_image, 
        tissue_color_classification, 
        wound_classification
    )

def extract_features_unet(image):
    # Load pre-trained U-Net model
    unet_model = tf.keras.models.load_model('unet_model.h5')
    
    # Preprocess image for U-Net input
    image = preprocess_image(image, unet_model.input_shape[1:])
    
    # Extract features using U-Net
    features = unet_model.predict(image)
    
    return features

def extract_features_rcnn(image):
    # Load pre-trained R-CNN model
    rcnn_model = tf.keras.models.load_model('rcnn_model.h5')
    
    # Preprocess image for R-CNN input
    image = preprocess_image(image, rcnn_model.input_shape[1:])
    
    # Extract features using R-CNN
    features = rcnn_model.predict(image)
    
    return features

def classify_tissue_color(features):
    # Load pre-trained tissue and color classification model
    classification_model = tf.keras.models.load_model('classification_model.h5')
    
    # Classify tissue and color
    classification = classification_model.predict(features)
    
    return classification

def classify_wound(features):
    # Load pre-trained wound classification model
    classification_model = tf.keras.models.load_model('wound_classification_model.h5')
    
    # Classify wound
    classification = classification_model.predict(features)
    
    return classification